In [17]:
from datetime import datetime
import pandas as pd
import numpy as np
# from src.models.avely_lstm import simple_lstm_v1, train_model, test_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Reshape, Input
from tensorflow.keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [2]:
df = pd.read_csv('../hiiu_liivi_data.csv')
df = df.dropna()
df = df[df['WindSpeed'] > 2]
df

,Datetime,0.084686,0.094604,0.104523,0.114441,0.124359,0.134277,0.144196,0.154114,0.164032,...,number,step,surface,valid_time,ERA5_Datetime,WindSpeed,MeanSeaLevelPressure,Temperature,Latitude,Longitude
0,2023-04-12 00:37:00,0.016039,0.015662,0.010723,0.010873,0.007623,0.010862,0.007251,0.003676,0.003972,...,0,0 days,0.0,2023-04-12 00:00:00,2023-04-12 01:00:00,6.546383,101208.420,278.89926,59.08,21.274
1,2023-04-12 00:59:00,0.198702,0.092199,0.024175,0.036656,0.058582,0.058957,0.028536,0.006233,0.007652,...,0,0 days,0.0,2023-04-12 00:00:00,2023-04-12 01:00:00,6.546383,101208.420,278.89926,59.08,21.274
2,2023-04-12 01:21:00,0.001898,0.001239,0.000962,0.000775,0.002440,0.001671,0.000502,0.001834,0.033752,...,0,0 days,0.0,2023-04-12 01:00:00,2023-04-12 01:00:00,6.546383,101208.420,278.89926,59.08,21.274
3,2023-04-12 01:43:00,0.001042,0.001539,0.001203,0.000404,0.000654,0.003298,0.000675,0.028470,0.017767,...,0,0 days,0.0,2023-04-12 01:00:00,2023-04-12 02:00:00,6.351250,101204.510,278.53810,59.33,21.024
4,2023-04-12 02:04:00,0.000872,0.001263,0.000459,0.000685,0.000260,0.004026,0.011854,0.009877,0.007712,...,0,0 days,0.0,2023-04-12 02:00:00,2023-04-12 02:00:00,6.351250,101204.510,278.53810,59.33,21.024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13945,2023-07-31 18:50:00,0.000719,0.000172,0.000289,0.000206,0.000301,0.000571,0.000519,0.002890,0.002223,...,0,0 days,0.0,2023-07-31 18:00:00,2023-07-31 19:00:00,7.475164,100350.984,291.05814,59.08,21.024
13946,2023-07-31 19:12:00,0.000428,0.000133,0.000429,0.000212,0.000265,0.001172,0.001698,0.002249,0.002895,...,0,0 days,0.0,2023-07-31 19:00:00,2023-07-31 19:00:00,7.475164,100350.984,291.05814,59.08,21.024
13947,2023-07-31 19:34:00,0.000521,0.000945,0.000268,0.000411,0.000485,0.001409,0.002094,0.002652,0.003487,...,0,0 days,0.0,2023-07-31 19:00:00,2023-07-31 20:00:00,7.371202,100351.960,290.91284,59.08,21.024
13948,2023-07-31 19:55:00,0.000333,0.000383,0.000238,0.000226,0.000800,0.001408,0.003670,0.003389,0.003109,...,0,0 days,0.0,2023-07-31 19:00:00,2023-07-31 20:00:00,7.371202,100351.960,290.91284,59.08,21.024


In [3]:

train_data = df.sample(frac=0.75, random_state=137)
test_data = df.drop(train_data.index)

train_feature = train_data.copy()
test_feature = test_data.copy()

train_labels = train_feature.pop('WindSpeed')
test_labels = test_feature.pop('WindSpeed')

train_feature = train_feature.drop(
    columns=['Datetime',
        'time',
             'latitude',
             'longitude',
             'u10',
             'v10',
             't2m',
             'msl',
             'meanSea',
             'number',
             'step',
             'surface',
             'valid_time',
             'ERA5_Datetime',
             'MeanSeaLevelPressure',
             'Temperature',
             'Latitude',
             'Longitude'])
test_feature = test_feature.drop(
    columns=['Datetime',
        'time',
             'latitude',
             'longitude',
             'u10',
             'v10',
             't2m',
             'msl',
             'meanSea',
             'number',
             'step',
             'surface',
             'valid_time',
             'ERA5_Datetime',
             'MeanSeaLevelPressure',
             'Temperature',
             'Latitude',
             'Longitude'])


In [4]:
data_train = np.log(train_feature.values)
data_test = np.log(test_feature.values)
infs_train = np.argwhere(np.isinf(data_train))
inf_test = np.argwhere(np.isinf(data_test))

infs_train_unique = np.unique(infs_train[:, 0])
infs_test_unique = np.unique(inf_test[:, 0])
infs_train_unique

C:\Users\Admin\AppData\Local\Temp\ipykernel_25944\3690231962.py:1: RuntimeWarning: divide by zero encountered in log
  data_train = np.log(train_feature.values)
C:\Users\Admin\AppData\Local\Temp\ipykernel_25944\3690231962.py:2: RuntimeWarning: divide by zero encountered in log
  data_test = np.log(test_feature.values)


array([ 619,  660,  867,  890,  902,  973, 1245, 1399, 1462, 1602, 1604,
       1664, 1789, 2035, 2081, 2267, 2447, 2816, 2934, 3145, 3390, 3418,
       3480, 3527, 3634, 3698, 3796, 4225, 4250, 4459, 4558, 4677, 5058,
       5575, 6259, 6487, 6616, 7004, 7063, 7440, 7799, 8352, 8384, 8484,
       8766, 9289, 9291])

In [5]:
train_labels.values

array([9.9215416 , 4.82027294, 5.80527029, ..., 3.23704917, 2.33800229,
       4.45706545])

In [6]:
# data_train = np.log(train_feature.values)
data_train = np.delete(data_train, infs_train_unique, axis=0)
data_train_reshape = data_train.reshape(data_train.shape[0], 1, data_train.shape[1])
label_train = np.delete(train_labels.values, infs_train_unique, axis=0)

# data_test = np.log(test_feature.values)
data_test = np.delete(data_test, infs_test_unique, axis=0)
data_test_reshape = data_test.reshape(data_test.shape[0], 1, data_test.shape[1])
label_test = np.delete(test_labels.values, infs_test_unique, axis=0)

In [7]:
print(data_train_reshape.shape, label_train.shape)

(9390, 1, 93) (9390,)


In [8]:
print(data_test_reshape.shape, label_test.shape)

(3122, 1, 93) (3122,)


# training

# Training

In [20]:
model = Sequential()
model.add(LSTM(64, input_shape=(1, data_train_reshape.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='tanh'))
model.add(Dense(1, activation='relu'))

model.compile(optimizer='adam', loss='mse')
model.summary()

C:\Users\sander.rikka\gitrepo\precipitation_prediction\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 1, 64)          │        40,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,897 (206.63 KB)

 Trainable params: 52,897 (206.63 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:

systime = datetime.now()
systimestr = datetime.strftime(systime, '%Y%m%dT%H%M')
mdl_name = f'{systimestr}_simple_lstm'

callbacks = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto', baseline=None, restore_best_weights=True)

# Train the model
history = model.fit(data_train_reshape,
                    label_train,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.3,
                    verbose=0)

results_df = pd.DataFrame(history.history)
results_csv_file = f'{mdl_name}_history.csv'
with open(results_csv_file, mode='w') as f:
   results_df.to_csv(f)

In [23]:
y_hat = model.predict(data_test_reshape, verbose=0)
accuracy = np.corrcoef(label_test, y_hat[:,0])[0, 1]
rmse = np.sqrt(np.mean((label_test - y_hat[:,0])**2))
print(f'Pearson correlation is {accuracy:.2f} and RMSE is {rmse:.2f}.')

Pearson correlation is 0.75 and RMSE is 1.65.
